Денис Иванов

Домашнее задание

На базе dataset'а Iris, только в части данных  по 2 классам : Iris Versicolor и Iris Virginica
реализовать самостоятельно логистическую регрессию и обучить ее методами: градиентный спуск, nesterov momentum,  rmsprop

In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, f1_score, recall_score


In [14]:
data = datasets.load_iris(as_frame=True).frame
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


Исключаем из датасета записи по сорту № 0 - Setosa?  приводим целевой признак к бинарной классификации (0/1)

In [15]:
datasets.load_iris().target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [16]:
data = data[data.target >0]
data.target = data.target - 1
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0


In [17]:
#  Нормализуем данные 
headers = list(data.iloc[:,:4].columns.values)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
data_scaled = min_max_scaler.fit_transform(data.iloc[:,:4])
data_norm = pd.DataFrame(data_scaled)
data_norm.columns = headers
data_norm.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.400000,0.333333,-0.128205,-0.466667
1,0.000000,0.333333,-0.230769,-0.333333
2,0.333333,0.222222,-0.025641,-0.333333
3,-0.600000,-0.666667,-0.487179,-0.600000
4,0.066667,-0.111111,-0.179487,-0.333333


In [72]:
model = LogisticRegression()
X = data_norm
y = data.target

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_tr = X_train.transpose()
y_train_tr = np.array(y_train).reshape(1,y_train.shape[0])
X_test_tr  = X_test.transpose()
y_test_tr  = np.array(y_test).reshape(1,y_test.shape[0])

Реализуем самостоятельно логистическую регрессию и сравним результаты со стандартным результатом работы функции sklearn.

In [74]:
class LogRegression():
    
    def __init__(self, num_iterations=1000, learning_rate=0.01, method='gb', gamma=None, eps=None):
        self.num_iterations = num_iterations               # количество итераций цикла оптимизации
        self.learning_rate  = learning_rate                # скорость обучения
        self.method         = method                       # метод оптимизации
        self.gamma          = gamma
        self.eps            = eps
        
    def sigmoid(self, x):
        s                   = 1.0 / (1.0 + np.exp(-x))
        return s

    def propagate(self, w, b, X, Y):                        # w - веса;  b - смещение, скаляр
        number_of_features  = X.shape[1]
        z                   = np.dot(w.T,X)+b
        A                   = self.sigmoid(z)
        dw                  = 1.0/number_of_features*np.dot(X, (A-Y).T) # градиент w
        db                  = 1.0/number_of_features*np.sum(A-Y)        # градиент b
        grads               = {"dw": dw, "db":db}
        return grads

    def optimize(self, X, Y, method):        
        w                  = np.zeros((X.shape[0],1))
        b                  = 0
        costs              = []   
        
        if self.method    == 'gb':
            for i in range(self.num_iterations):
                grads      = self.propagate(w, b, X, Y)
                dw         = grads["dw"]
                db         = grads["db"]
                w          = w - self.learning_rate*dw
                b          = b - self.learning_rate*db
        
        elif self.method  == 'nm':
            v_w            = 0
            v_b            = 0
            for i in range(self.num_iterations):
                grads      = self.propagate(w, b, X, Y)
                dw         = grads["dw"]
                db         = grads["db"]
                v_w        = self.gamma*v_w + self.learning_rate*(1-self.gamma)*dw             
                v_b        = self.gamma*v_b + self.learning_rate*(1-self.gamma)*db                
                w          = w - v_w
                b          = b - v_b
                
        elif self.method  == 'rmsp':
            EG_w = 0
            EG_b = 0
            for i in range(self.num_iterations):
                grads      = self.propagate(w, b, X, Y)
                dw         = grads["dw"]
                db         = grads["db"]
                EG_w       = self.gamma*EG_w + (1-self.gamma)*(dw/self.learning_rate)**2                
                EG_b       = self.gamma*EG_b + (1-self.gamma)*(db/self.learning_rate)**2
                w          = w - dw/(EG_w + self.eps)**0.5
                b          = b - db/(EG_b + self.eps)**0.5

        grads = {"dw": dw, "db": db}
        params = {"w": w, "b": b}
        return params
        
  
    def predict(self, w, b, X):        # Предсказывает будет значение 0 или 1 (граница 0.5).
        y_prediction     = np.zeros((1,X.shape[1]))
        w                = w.reshape(X.shape[0],1)
        A                = self.sigmoid(np.dot(w.T, X) + b)

        for i in range(A.shape[1]):
            if (A[:,i] > 0.5): 
                y_prediction[:, i] = 1
            elif (A[:,i] <= 0.5):
                y_prediction[:, i] = 0
        return y_prediction[0]
    
    def fit_predict(self, X_train, y_train, X_test):
        parameters      = self.optimize(X_train, y_train, self.method)
        w               = parameters["w"]
        b               = parameters["b"]
        y_test_predict  = self.predict(w, b, X_test)
        return y_test_predict

In [75]:
lr_nm               = LogRegression(num_iterations=1000, learning_rate=0.001, method='nm',   gamma=0.85)
lr_rmsp             = LogRegression(num_iterations=1000, learning_rate=0.001, method='rmsp', gamma=0.8, eps=1e-6)
lr_gb               = LogRegression(num_iterations=1000, learning_rate=0.001, method='gb'  , gamma=0.95)

y_test_predict_nm   = lr_nm.fit_predict  (X_train_tr, y_train_tr, X_test_tr)
y_test_predict_rmsp = lr_rmsp.fit_predict(X_train_tr, y_train_tr, X_test_tr)
y_test_predict_gb   = lr_gb.fit_predict  (X_train_tr, y_train_tr, X_test_tr)

In [76]:
model.fit(X_train, y_train)
y_test_predict_st   = model.predict(X_test)

In [77]:
index           = ['lr_nm', 'lr_rmsp', 'lr_gb', 'lr_standart']
metrics_columns = ['ROC_AUC', 'RECALL', 'ACCURACY', 'PRECISION', 'F1']
y_preds         = [ y_test_predict_nm, y_test_predict_rmsp, y_test_predict_gb, y_test_predict_st]
metrics_scores  = np.zeros(20).reshape(4,5)

In [78]:
def metrics(y_test, y_pred, metric_type):
    if metric_type == 'ROC_AUC':
        return roc_auc_score(y_test, y_pred)    
    elif metric_type == 'RECALL':
        return recall_score(y_test, y_pred)    
    elif metric_type == 'ACCURACY':
        return accuracy_score(y_test, y_pred)    
    elif metric_type == 'PRECISION':
        return precision_score(y_test, y_pred)    
    elif metric_type == 'F1':
        return f1_score(y_test, y_pred)

In [79]:
for i in range(0, len(index)):  
    for k in range(0, len(metrics_columns)):
        metrics_scores[i][k] = metrics(y_test, y_preds[i], metric_type=metrics_columns[k])

In [80]:
metrics_df = pd.DataFrame(metrics_scores, index=index, columns=metrics_columns)
metrics_df

,ROC_AUC,RECALL,ACCURACY,PRECISION,F1
lr_nm,0.909091,0.818182,0.90,1.0,0.900000
lr_rmsp,0.954545,0.909091,0.95,1.0,0.952381
lr_gb,0.909091,0.818182,0.90,1.0,0.900000
lr_standart,0.954545,0.909091,0.95,1.0,0.952381
